# Производительность модели ключ-значение на примере работы с Redis

<p> https://en.wikipedia.org/wiki/Redis
<p> https://github.com/andymccurdy/redis-py

```python
'css:key': {
    "reg_id": <string>,
    "reg_name": <string>,
    "year": <number>,
    "group_id": <number>,
    "group_name": <string>,
    "category": <string>,
    "gender": <string>,
    "value": <number>,
}
```

In [1]:
! redis-server --version

Redis server v=4.0.9 sha=00000000:0 malloc=jemalloc-4.0.3 bits=64 build=3bcf075214098f11


In [2]:
import redis

In [3]:
pool = redis.ConnectionPool(decode_responses=True)
r = redis.StrictRedis(connection_pool=pool)

## 1. Получение множества всех полей по ключу хеша

In [4]:
r.hgetall('css:0')

{'reg_id': '1100',
 'reg_name': 'Российская Федерация',
 'year': '2016',
 'group_id': '14002',
 'group_name': 'по возрасту',
 'category': '14-15',
 'gender': 'Мужчины',
 'value': '13573'}

## 2. Получение множества значений по ключу хеша

In [5]:
r.hvals('css:0')

['1100',
 'Российская Федерация',
 '2016',
 '14002',
 'по возрасту',
 '14-15',
 'Мужчины',
 '13573']

## 3. Получение значений определенных значений в множестве по ключу хеша

In [6]:
r.hmget('css:0', 'reg_id', 'reg_name')

['1100', 'Российская Федерация']

In [7]:
%timeit r.hmget('css:0', 'reg_id', 'reg_name')

447 µs ± 127 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 4. Сохранение в хеш пар ключ-значение

In [8]:
data = {
    'reg_id': '1100',
    'reg_name': 'Российская Федерация',
    'year': '2018',
    'group_id': '14002',
    'group_name': 'по возрасту',
    'category': '14-15',
    'gender': 'Мужчины',
    'value': '1'
}
i = int(1e7) - 1
r.hmset('css:{}'.format(i), data)

True

In [9]:
i += 1
def foo():
    global i, data
    r.hmset('css:{}'.format(i), data)
    i += 1

In [10]:
%timeit -n 1000 -r 5 foo()

643 µs ± 117 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


## 5. Обновление значений по ключу хеша

In [11]:
i = int(1e7) - 1
r.hmset('css:{}'.format(i), {'year': '2017', 'value': '1'})

True

In [12]:
i += 1
def foo():
    global i
    r.hmset('css:{}'.format(i), {'year': '2017', 'value': '1'})
    i += 1

In [13]:
%timeit -n 1000 -r 5 foo()

405 µs ± 83.3 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


## 6. Удаление хеша

In [14]:
i = int(1e7) - 1
fields = r.hkeys('css:{}'.format(i))
r.hdel('css:{}'.format(i), fields)

0

In [15]:
i += 1
def foo():
    global i, fields
    r.hdel('css:{}'.format(i), fields)
    i += 1

In [16]:
%timeit -n 1000 -r 5 foo()

356 µs ± 45 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)
